# Programming with RDDs -1

#### RDD basics

RDD - Resilient Distributed Dataset

Resilient - anything that can be recreated

In [4]:
sc

In [5]:
from pyspark import SparkContext as sc2

In [6]:
sc2

pyspark.context.SparkContext

In [43]:
linesRDD = sc.textFile("file:///home/cloudera/Desktop/pyspark/README.txt")

In [44]:
linesRDD.first()

u'This is a sample file'

In [47]:
#Transformation
pythonLinesRDD = linesRDD.filter(lambda line : 'Python' in line)

In [48]:
#Action
pythonLinesRDD.first()

u'Python is a langauge developed by Guido Van Rosam'

In [51]:
#parallelize() -  only used in prototyping, testing and learning.
testRDD = sc.parallelize(['Spark','is','very','powerful']) 

In [52]:
testRDD

ParallelCollectionRDD[36] at parallelize at PythonRDD.scala:423

#### RDD Operations

###### Transformations - returns a new RDD

example - map() , filter(), flatMap(), distinct(), sample()

###### Actions -returns other data type but not RDD

example - count(), first(),top(),collect()

In [54]:
testRDD.collect()

['Spark', 'is', 'very', 'powerful']

<b>collect</b> should <b>not</b> be used on large data , as it tries to load the entire data in driver.

instead we can use take() or store the RDD in HDFS or S3.

#### Lazy Evaluation

Transformations on RDD are not executed untill the rdd is called by an Action.

<u>example:</u> so when we call aRDD = sc.textFile('abc.txt') , Spark  does not load the file ,untill aRDD is called by an action .

<b><u>How Lazy Evaluation is useful</u></b>

This makes lot of sense when we deal with BIG Data , For example in line #43 , the entire data is not loaded , when we call first() it only read the first line rather than going through the entire file.

Lazy evaluation reduces the number of passes it has to to take over our data by grouping operations together .


##### Passing Functions to Spark

In [56]:
#Three ways

In [57]:
#1. Passing functions
pythonLinesRDD = linesRDD.filter(lambda line : 'Python' in line)

In [60]:
#same as earlier

def cotainsPython(s):
    return 'Pyhton' in s
pythonLinesRDD = linesRDD.filter(cotainsPython)

In [62]:
#Passing a function with feild refrence
#<b>NEVER DO THIS</b>
class SearchFunctions(object):
    def __init__(self,query):
        self.query=query
    def isMatch(self, s):
        return self.query in s
    
    def getMatchesFunctionReference(self,rdd):
        #Problem : references all of the self object
        return rdd.filter(self.isMatch)
    def getMatchesMemberReference(self,rdd):
        # Problem : refernces all of the "self" in self.query
        return rdd.filter(lambda x: self.query in x)


<b>Important<b>:The problem with the above eaxmple is that Spark sends entire object to  worker nodes, which can be much larger than the information we need.

In [63]:
#Passing without feild references

class wordFunctions(object):
    
    def getMatchesNoRefrence(self, rdd):
        #Safe : extract only the feild we need into a local variable
        query = self.query
        return rdd.filter(lambda x:query in x)